In [ ]:
!pip install shap

In [ ]:
!pip install rdkit-pypi

In [ ]:
!pip install shap rdkit scikit-learn

In [ ]:
pip install bayesian-optimization

In [ ]:
import pandas as pd
import numpy as np
from rdkit import Chem  # Ensure RDKit's Chem module is imported
from rdkit.Chem import rdFingerprintGenerator
from rdkit import DataStructs
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, RandomizedSearchCV, cross_val_predict, train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, explained_variance_score, make_scorer
import matplotlib.pyplot as plt
from scipy.stats import randint, uniform
import seaborn as sns
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import cross_val_score
from sklearn.inspection import permutation_importance
from sklearn.inspection import PartialDependenceDisplay
from sklearn.svm import SVR
import math
from scipy.stats import uniform
import BayesianOptimization 
from bayes_opt import BayesianOptimization
import shap
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import randint

In [ ]:
# Load the dataset
glp = pd.read_csv("glp.csv")

In [ ]:
# Initial data exploration
print(glp.head())
print(glp.tail())
print(glp.describe())
print(glp.shape)
print(glp.isnull().sum())

In [ ]:
# Early EDA
plt.figure(figsize=(10, 6))
sns.histplot(glp['pChEMBL Value'], kde=True)
plt.title('Distribution of pChEMBL Values')
plt.xlabel('pChEMBL Value')
plt.ylabel('Frequency')
plt.show()

plt.figure(figsize=(6, 8))
sns.boxplot(y=glp['pChEMBL Value'])
plt.title('Boxplot of pChEMBL Values')
plt.ylabel('pChEMBL Value')
plt.show()

In [ ]:
# Extract SMILES and pChEMBL values
smiles = glp['Smiles']
pchembl_value = glp['pChEMBL Value']

# Function to convert SMILES to molecular fingerprints
def smiles_to_fingerprint(smiles):
    """Converts a SMILES string to a molecular fingerprint."""
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(2048)  # Handle invalid SMILES
    generator = rdFingerprintGenerator.GetMorganGenerator(radius=2)
    fp = generator.GetFingerprint(mol)
    arr = np.zeros((2048,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    return arr

In [ ]:
# Apply the function to the SMILES column
X = np.array([smiles_to_fingerprint(sm) for sm in smiles])

# Define the target
y = pchembl_value

In [ ]:
# Feature scaling
scaler_X = RobustScaler()
X_scaled = scaler_X.fit_transform(X)

In [ ]:
# Set up KFold cross-validation
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Define a function to compute the metrics
def evaluate_model(model, X, y, cv):
    predictions = cross_val_predict(model, X, y, cv=cv)
    rmse = np.sqrt(mean_squared_error(y, predictions))
    mae = mean_absolute_error(y, predictions)
    r2 = r2_score(y, predictions)
    ev = explained_variance_score(y, predictions)
    print(f"RMSE: {rmse:.4f}, MAE: {mae:.4f}, R²: {r2:.4f}, EV: {ev:.4f}")
    return rmse, mae, r2, ev

In [ ]:
### Multi-Layer Perceptron (MLP) Regressor with Randomized Search ###
mlp = MLPRegressor(max_iter=500, random_state=42)
param_dist_mlp = {
    'hidden_layer_sizes': [(randint.rvs(50, 150),), (randint.rvs(100, 300),), (randint.rvs(150, 500),)],
    'activation': ['logistic', 'relu', 'tanh'],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'alpha': uniform(0.0001, 0.1),
    'learning_rate': ['constant', 'adaptive', 'invscaling'],
    'learning_rate_init': uniform(0.0001, 0.01),
    'max_iter': randint(100, 1000),
    'early_stopping': [True, False],
}
random_search_mlp = RandomizedSearchCV(mlp, param_distributions=param_dist_mlp, n_iter=50, cv=5, verbose=2, random_state=42, n_jobs=-1)
random_search_mlp.fit(X_scaled, y)

In [ ]:
# Evaluate MLP model
print("MLP Regressor Metrics:")
evaluate_model(random_search_mlp.best_estimator_, X_scaled, y, kf)

In [ ]:
### Support Vector Machine (SVR) with Randomized Search ###
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)
y_train_scaled = scaler_y.fit_transform(y_train.values.reshape(-1, 1)).flatten()

svr = SVR(kernel='rbf', C=1.0, epsilon=0.1)
svr.fit(X_train_scaled, y_train_scaled)

In [ ]:
# Evaluate SVR model
print("SVR Metrics:")
evaluate_model(svr, X_scaled, y, kf)

In [ ]:
### Random Forest Regressor with Randomized Search ###
rf = RandomForestRegressor(random_state=42)
param_dist_rf = {
    'n_estimators': randint(50, 500),
    'max_depth': randint(3, 20),
    'min_samples_split': randint(2, 50),
    'min_samples_leaf': randint(1, 20),
    'bootstrap': [True, False],
    'max_features': randint(1, X_scaled.shape[1])
}
random_search_rf = RandomizedSearchCV(rf, param_distributions=param_dist_rf, n_iter=50, cv=5, verbose=2, random_state=42, n_jobs=-1)
random_search_rf.fit(X_scaled, y)

In [ ]:
# Evaluate Random Forest model
print("Random Forest Metrics:")
evaluate_model(random_search_rf.best_estimator_, X_scaled, y, kf)

In [ ]:
# Model evaluation and SHAP explanations

### Random Forest Model Evaluation and SHAP ###
explainer_rf = shap.TreeExplainer(random_search_rf.best_estimator_)
shap_values_rf = explainer_rf.shap_values(X_scaled)

# SHAP summary plot for Random Forest
shap.summary_plot(shap_values_rf, X_scaled, feature_names=[f'Feature {i}' for i in range(X_scaled.shape[1])], plot_type="bar")
shap.summary_plot(shap_values_rf, X_scaled, feature_names=[f'Feature {i}' for i in range(X_scaled.shape[1])])

In [ ]:
# Predicted vs Actual plot for Random Forest
y_pred_rf = random_search_rf.best_estimator_.predict(X_scaled)
plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred_rf)
plt.plot([min(y), max(y)], [min(y), max(y)], color='red', linestyle='--')
plt.title('Predicted vs Actual pChEMBL Values (Random Forest)')
plt.xlabel('Actual pChEMBL Value')
plt.ylabel('Predicted pChEMBL Value')
plt.show()

# Residual plot for Random Forest
residuals_rf = y - y_pred_rf
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_rf, residuals_rf)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residual Plot (Random Forest)')
plt.xlabel('Predicted pChEMBL Value')
plt.ylabel('Residuals')
plt.show()

In [ ]:
### MLP Model Evaluation and SHAP ###
explainer_mlp = shap.KernelExplainer(random_search_mlp.best_estimator_.predict, X_scaled)
shap_values_mlp = explainer_mlp.shap_values(X_scaled, nsamples=100)

# SHAP summary plot for MLP
shap.summary_plot(shap_values_mlp, X_scaled, feature_names=[f'Feature {i}' for i in range(X_scaled.shape[1])], plot_type="bar")
shap.summary_plot(shap_values_mlp, X_scaled, feature_names=[f'Feature {i}' for i in range(X_scaled.shape[1])])

In [ ]:
# Predicted vs Actual plot for MLP
y_pred_mlp = random_search_mlp.best_estimator_.predict(X_scaled)
plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred_mlp)
plt.plot([min(y), max(y)], [min(y), max(y)], color='red', linestyle='--')
plt.title('Predicted vs Actual pChEMBL Values (MLP)')
plt.xlabel('Actual pChEMBL Value')
plt.ylabel('Predicted pChEMBL Value')
plt.show()

# Residual plot for MLP
residuals_mlp = y - y_pred_mlp
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_mlp, residuals_mlp)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residual Plot (MLP)')
plt.xlabel('Predicted pChEMBL Value')
plt.ylabel('Residuals')
plt.show()

In [ ]:
### SVR Model Evaluation and SHAP ###
explainer_svr = shap.KernelExplainer(svr.predict, X_train_scaled)
shap_values_svr = explainer_svr.shap_values(X_scaled, nsamples=100)

# SHAP summary plot for SVR
shap.summary_plot(shap_values_svr, X_scaled, feature_names=[f'Feature {i}' for i in range(X_scaled.shape[1])], plot_type="bar")
shap.summary_plot(shap_values_svr, X_scaled, feature_names=[f'Feature {i}' for i in range(X_scaled.shape[1])])

In [ ]:
# Predicted vs Actual plot for SVR
y_pred_svr = svr.predict(X_scaled)
plt.figure(figsize=(10, 6))
plt.scatter(y, y_pred_svr)
plt.plot([min(y), max(y)], [min(y), max(y)], color='red', linestyle='--')
plt.title('Predicted vs Actual pChEMBL Values (SVR)')
plt.xlabel('Actual pChEMBL Value')
plt.ylabel('Predicted pChEMBL Value')
plt.show()

# Residual plot for SVR
residuals_svr = y - y_pred_svr
plt.figure(figsize=(10, 6))
plt.scatter(y_pred_svr, residuals_svr)
plt.axhline(0, color='red', linestyle='--')
plt.title('Residual Plot (SVR)')
plt.xlabel('Predicted pChEMBL Value')
plt.ylabel('Residuals')
plt.show()

In [ ]:
    """
    Predictions for IIG Dataset
    """

In [ ]:
# Load the IIG dataset
IIG_df = pd.read_csv("")
iig_smiles = IIG_df['CanonicalSMILES']

In [ ]:
# Convert SMILES to molecular fingerprints
iig_fingerprints = np.array([smiles_to_fingerprint(sm) for sm in iig_smiles])

# Scale the fingerprints using the same scaler used for training
iig_scaled = scaler_X.transform(iig_fingerprints)

In [ ]:
    """
    Predictions for each model
    """

In [ ]:
# Random Forest (RF) Predictions
iig_predictions_rf = random_search_rf.best_estimator_.predict(iig_scaled)
IIG_df['Predicted pChEMBL Value_rf'] = iig_predictions_rf

In [ ]:
# SVR Predictions
iig_predictions_svr = svr.predict(iig_scaled)
IIG_df['Predicted pChEMBL Value_svr'] = iig_predictions_svr

In [ ]:
# MLP Predictions
iig_predictions_mlp = random_search_mlp.best_estimator_.predict(iig_scaled)
IIG_df['Predicted pChEMBL Value_mlp'] = iig_predictions_mlp

In [ ]:
# Save the predictions to a new CSV file for each model
IIG_df.to_csv('~/Documents/IIG_predictions_rf_svr_mlp.csv', index=False)
print("Predictions for IIG dataset saved.")

In [ ]:
    """
    Top 3 Excipients for Each Model 
    Based on Predicted pChEMBL Values
    """

In [ ]:
# For Random Forest
top_3_excipients_rf = IIG_df.nlargest(3, 'Predicted pChEMBL Value_rf')[['INGREDIENT_NAME', 'Predicted pChEMBL Value_rf']]
print("Top 3 Excipients for RF:")
print(top_3_excipients_rf)

In [ ]:
# For SVR
top_3_excipients_svr = IIG_df.nlargest(3, 'Predicted pChEMBL Value_svr')[['INGREDIENT_NAME', 'Predicted pChEMBL Value_svr']]
print("Top 3 Excipients for SVR:")
print(top_3_excipients_svr)

In [ ]:
# For MLP
top_3_excipients_mlp = IIG_df.nlargest(3, 'Predicted pChEMBL Value_mlp')[['INGREDIENT_NAME', 'Predicted pChEMBL Value_mlp']]
print("Top 3 Excipients for MLP:")
print(top_3_excipients_mlp)

In [ ]:
    """
    Creating boxplots for each model's predicted pChEMBL values
    """

In [ ]:
# Boxplot for Random Forest
plt.figure(figsize=(10, 6))
plt.boxplot(IIG_df['Predicted pChEMBL Value_rf'])
plt.title('Boxplot of Predicted pChEMBL Values (Random Forest)')
plt.ylabel('Predicted pChEMBL Value_rf')
plt.show()

# Boxplot for SVR
plt.figure(figsize=(10, 6))
plt.boxplot(IIG_df['Predicted pChEMBL Value_svr'])
plt.title('Boxplot of Predicted pChEMBL Values (SVR)')
plt.ylabel('Predicted pChEMBL Value_svr')
plt.show()

# Boxplot for MLP
plt.figure(figsize=(10, 6))
plt.boxplot(IIG_df['Predicted pChEMBL Value_mlp'])
plt.title('Boxplot of Predicted pChEMBL Values (MLP)')
plt.ylabel('Predicted pChEMBL Value_mlp')
plt.show()